In [1]:
from glob import glob
from tqdm import tqdm
from unidecode import unidecode
from sklearn.feature_extraction.text import CountVectorizer
import json
import os
import mp
import re
import string

timestamps = [i * 0.02 for i in range(1500 + 1)]

rejected_c = set(string.digits) | set(string.punctuation)
def clean(s):
    for r in rejected_c:
        s = s.replace(r, ' ')
    return re.sub(r'[ ]+', ' ', ''.join(s)).strip()

In [2]:
from huggingface_hub import snapshot_download
snapshot_download(
    repo_id="mesolitica/malay-dialect-dictionary-instructions", 
    repo_type = 'dataset',
    local_dir = './malay-dialect-dictionary-instructions')

Fetching 11 files:   0%|          | 0/11 [00:00<?, ?it/s]

'/home/husein/ssd3/malay-dialect-dictionary-instructions'

In [3]:
keywords = set()

for f in tqdm(glob('malay-dialect-dictionary-instructions/*jsonl')):
    with open(f) as fopen:
        for l in fopen:
            l = json.loads(l)
            k = clean(unidecode(l['left'].lower())).split()
            for k_ in k:
                keywords.add(k_)
len(keywords)

100%|██████████████████████████████████████████████████████████████████████████████████████| 10/10 [00:00<00:00, 101.50it/s]


5979

In [4]:
files = glob('dialects_processed/**/*/*.json', recursive = True)

len(files)

147181

In [5]:
rejected = [
    'terima kasih kerana menonton',
    'terima kasih',
    'thank you for watching',
]

In [9]:
def new_path(f):
    f = f.replace('.mp3', '.alignment')
    f = f.replace('_processed/', '_processed_alignment/')
    return f

def loop(files):
    files, _ = files
    data = []
    for file in tqdm(files):
        folder = os.path.split(file)[0]
        folder_folder = os.path.split(folder)[1]
        filename = file.replace('.json', '')

        try:
            with open(file) as fopen:
                d = json.load(fopen)
        except:
            continue

        for no, obj in enumerate(d):
            text = obj["text"].strip()
            
            rt_ = re.sub('[^a-z ]+', '', text.lower()).strip()
            if any([s == rt_ for s in rejected]):
                continue
            
            if not len(set(clean(text.lower()).split()) & keywords):
                continue
            
            try:
                dense = CountVectorizer(ngram_range = (3,3)).fit_transform([text]).todense()
                repeat = (dense > 3).sum() >= 1
                if repeat:
                    continue
            except:
                continue
            
            audio_path = os.path.join(folder, f'{folder_folder}_{no}.mp3')
            align_path = new_path(audio_path)
            
            if not os.path.exists(align_path):
                continue
                
            with open(align_path) as fopen:
                align = json.load(fopen)
                
            scores = [a for a in align if a['score'] <= -15]
            if len(scores):
                continue

            data.append({
                'audio': audio_path,
                'alignment': align_path,
            })
    
    return data

In [10]:
d = loop((files[-10:], 0))

100%|██████████████████████████████████████████████████████████████████████████████████████| 10/10 [00:00<00:00, 623.87it/s]


In [11]:
d[0]

{'audio': 'dialects_processed/‘Zaman PN harga barang naik mencanak!’ - Rafizi [OjJ6hCKU1hE]/‘Zaman PN harga barang naik mencanak!’ - Rafizi [OjJ6hCKU1hE]_0.mp3',
 'alignment': 'dialects_processed_alignment/‘Zaman PN harga barang naik mencanak!’ - Rafizi [OjJ6hCKU1hE]/‘Zaman PN harga barang naik mencanak!’ - Rafizi [OjJ6hCKU1hE]_0.alignment'}

In [12]:
data = mp.multiprocessing(files, loop, cores = 20)
len(data)

100%|███████████████████████████████████████████████████████████████████████████████████| 7359/7359 [01:54<00:00, 64.08it/s]


1131225

In [21]:
d_ = data[1]

In [26]:
d_

{'audio': 'dialects_processed/KILAUAN EMAS ZON UTARA  KEPUTUSAN [tVjxnbDuEfE]/KILAUAN EMAS ZON UTARA  KEPUTUSAN [tVjxnbDuEfE]_1.mp3',
 'alignment': 'dialects_processed_alignment/KILAUAN EMAS ZON UTARA  KEPUTUSAN [tVjxnbDuEfE]/KILAUAN EMAS ZON UTARA  KEPUTUSAN [tVjxnbDuEfE]_1.alignment'}

In [24]:
segments

[[{'start': 0.08, 'end': 0.44, 'text': 'Akak', 'score': -0.01957535743713379},
  {'start': 0.58, 'end': 0.76, 'text': 'yang', 'score': -0.21853363513946533},
  {'start': 0.88,
   'end': 1.38,
   'text': 'berbaju,',
   'score': -0.09531104564666748}],
 [{'start': 2.9, 'end': 3.08, 'text': 'pink', 'score': -0.2573007345199585},
  {'start': 3.14, 'end': 3.2, 'text': 'ni', 'score': -0.030910491943359375},
  {'start': 3.26, 'end': 3.38, 'text': 'lah,', 'score': -0.01957535743713379},
  {'start': 3.46, 'end': 3.52, 'text': 'saya', 'score': -12.183582305908203},
  {'start': 3.6, 'end': 3.68, 'text': 'nak', 'score': -0.4333171844482422},
  {'start': 3.72, 'end': 3.9, 'text': 'tanya', 'score': -4.709495544433594},
  {'start': 4.0, 'end': 4.04, 'text': 'ni.', 'score': -0.03769683837890625}],
 [{'start': 5.14, 'end': 5.32, 'text': 'Akak', 'score': -0.5163669586181641},
  {'start': 5.36, 'end': 5.52, 'text': 'rasa', 'score': -8.338333129882812},
  {'start': 5.56, 'end': 5.68, 'text': 'akak', 'scor

In [25]:
import IPython.display as ipd
ipd.Audio(d_['audio'])

In [31]:
def loop(rows):
    rows, _ = rows
    result = []
    for r in tqdm(rows):
        f = r['audio']
        
        with open(r['alignment']) as fopen:
            align = json.load(fopen)

        segments, temp = [], [align[0]]
        last_t = align[0]['end']
        for c in align[1:]:
            if (c['start'] - last_t) > 0.25:
                segments.append(temp)
                temp = []

            last_t = c['end']
            temp.append(c)

        if len(temp):
            segments.append(temp)

        ts = []
        for s in segments:
            start = min(timestamps, key=lambda t: abs(t - s[0]['start']))
            end = min(timestamps, key=lambda t: abs(t - s[-1]['end']))
            w = ' '.join([c['text'] for c in s])
            t = f"<|{start:.2f}|> {w}<|{end:.2f}|>"
            ts.append(t)

        ts = ''.join(ts)
        new_text = text = f"<|startoftranscript|><|ms|><|transcribe|>{ts}<|endoftext|>"
        
        result.append({
            'new_text': new_text,
            'audio_filename': f
        })
    return result

In [32]:
r = loop((data[:100], 0))

100%|███████████████████████████████████████████████████████████████████████████████████| 100/100 [00:00<00:00, 1753.24it/s]


In [33]:
r[0]

{'new_text': '<|startoftranscript|><|ms|><|transcribe|><|0.04|> Abang yakin tak,<|0.88|><|1.14|> untuk<|1.58|><|1.90|> mewakili Zon Utara?<|3.16|><|endoftext|>',
 'audio_filename': 'dialects_processed/KILAUAN EMAS ZON UTARA  KEPUTUSAN [tVjxnbDuEfE]/KILAUAN EMAS ZON UTARA  KEPUTUSAN [tVjxnbDuEfE]_0.mp3'}

In [34]:
r[-1]

{'new_text': '<|startoftranscript|><|ms|><|transcribe|><|0.02|> Kita<|0.34|><|0.78|> yang transfer tu,<|1.40|><|1.74|> tajuknya mesti bahagian nombor satu ni. Ni paling penting.<|4.48|><|5.24|> Kalau tajuk tu duduk bahagian nombor dua,<|7.24|><|8.46|> kata studio tak boleh baca.<|9.96|><|endoftext|>',
 'audio_filename': 'dialects_processed/🔴[LIVE] ASAS DATA STUDIO ｜ SIRI 1 [rpM7zMlbwEg]/🔴[LIVE] ASAS DATA STUDIO ｜ SIRI 1 [rpM7zMlbwEg]_139.mp3'}

In [35]:
r = mp.multiprocessing(data, loop, cores = 15)

100%|████████████████████████████████████████████████████████████████████████████████| 75415/75415 [01:25<00:00, 879.13it/s]


In [36]:
import pandas as pd

pd.DataFrame(r).to_parquet('gather-dialects-segment.parquet')

In [37]:
from huggingface_hub import HfApi
api = HfApi()
api.upload_file(
    path_or_fileobj="gather-dialects-segment.parquet",
    path_in_repo="data/dialects_segment-00000-of-00001.parquet",
    repo_id="malaysia-ai/STT-Whisper",
    repo_type="dataset",
)

gather-dialects-segment.parquet:   0%|          | 0.00/159M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/datasets/malaysia-ai/STT-Whisper/commit/a6b36d3f0f754dfd569680f0c9c16e55a8c26150', commit_message='Upload data/dialects_segment-00000-of-00001.parquet with huggingface_hub', commit_description='', oid='a6b36d3f0f754dfd569680f0c9c16e55a8c26150', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/malaysia-ai/STT-Whisper', endpoint='https://huggingface.co', repo_type='dataset', repo_id='malaysia-ai/STT-Whisper'), pr_revision=None, pr_num=None)